# HDA - Project 3
## TASK B1: Activity detection
This task consists of a binary classification, where a gesture denotes activity and thus the model detects wheter there is a gesture label or not (labeled in column 6).

This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
subject = 1
folder = "./data/full/"
label = 0     # default for task B1
window_size = 15
stride = 15
# make_binary = True

In [2]:
import utils
import deeplearning
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


Creation of training set and test set

In [3]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                                folder,
                                                                                label,
                                                                                window_size,
                                                                                stride,
                                                                                printInfo = True,
                                                                                make_binary = True)

Training samples:  157125 
Test samples:       57536 
Features:             110
Classes: 2

TRAINING SET:
Dataset of Images have shape:  (10474, 15, 110) 
Dataset of Labels have shape:    (10474, 2) 
Fraction of labels:   [0.11065496 0.88934504]

TEST SET:
Dataset of Images have shape:  (3834, 15, 110) 
Dataset of Labels have shape:    (3834, 2) 
Fraction of labels:   [0.17736046 0.82263954]


Preparation of data in a input-suitable form

In [4]:
detection_model = deeplearning.MotionDetection((window_size,n_features,1), n_classes)
detection_model.summary() # model visualization

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 15, 110, 1)        4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 108, 50)        1700      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 108, 50)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 5400)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 20)             433680    
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               10752     
__________

In [5]:
def f1_loss(y_true, y_pred):
    return 1 - utils.f1_score(np.argmax(y_true), y_pred,average='weighted')

In [6]:
detection_model.compile(optimizer = Adam(lr=0.01), 
                        loss = "categorical_crossentropy",
                        metrics = ["accuracy"])
                        

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_d.hdf5', verbose=1, save_best_only=True)

detection_model.fit(x = input_train, 
                    y = y_train, 
                    epochs = 5, 
                    batch_size = 128,
                    verbose = 1,
                    validation_data=(input_test, y_test),
                    callbacks=[checkpointer])

Train on 10474 samples, validate on 3834 samples
Epoch 1/5
10474/10474 [==============================] - 6s 566us/step - loss: 0.2239 - acc: 0.9210 - val_loss: 0.3154 - val_acc: 0.9171

Epoch 00001: val_loss improved from inf to 0.31538, saving model to ./data/weights_d.hdf5
Epoch 2/5
10474/10474 [==============================] - 2s 235us/step - loss: 0.1674 - acc: 0.9488 - val_loss: 0.2032 - val_acc: 0.9296

Epoch 00002: val_loss improved from 0.31538 to 0.20319, saving model to ./data/weights_d.hdf5
Epoch 3/5
10474/10474 [==============================] - 2s 235us/step - loss: 0.1574 - acc: 0.9485 - val_loss: 0.2793 - val_acc: 0.9152

Epoch 00003: val_loss did not improve
Epoch 4/5
10474/10474 [==============================] - 3s 264us/step - loss: 0.1536 - acc: 0.9547 - val_loss: 0.1993 - val_acc: 0.9314

Epoch 00004: val_loss improved from 0.20319 to 0.19930, saving model to ./data/weights_d.hdf5
Epoch 5/5
10474/10474 [==============================] - 3s 273us/step - loss: 0.18

In [7]:
y_pred = detection_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.84      0.71      0.77       680
          1       0.94      0.97      0.95      3154

avg / total       0.92      0.92      0.92      3834



In [8]:
detection_model_best = load_model('./data/weights_d.hdf5')

y_pred = detection_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

             precision    recall  f1-score   support

          0       0.81      0.80      0.81       680
          1       0.96      0.96      0.96      3154

avg / total       0.93      0.93      0.93      3834



## Task B2: gesture recognition
This task consists of a 17-class classification, where gestures are labeled in column 6.

To tune the following parameters, refer to the first cell of task B1:
- subject: select the subject on which to test the model, between [1,4];
- folder: directory name where '.mat' files are stored;
- label_col: index of feature column to be selected to perform activity detection, between [0,6];
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

Here we just need to preserve the different labels, thus we set 'make_binary' to False. We have then 18 different labels, keeping into account the null class, with label 0.

In [ ]:
[x_train, y_train, x_test, y_test, n_classes] = utils.preprocessing(subject,
                                                                    folder,
                                                                    label,
                                                                    window_size,
                                                                    stride,
                                                                    printInfo = True,
                                                                    make_binary = False,
                                                                    null_class = False)

In [ ]:
n_classes = 4 # OVERWRITE TO BE FIXED

In [ ]:
classification_model = deeplearning.MotionClassification((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

In [ ]:
classification_model.compile(optimizer = Adam(lr=0.01), 
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_c.hdf5', verbose=1, save_best_only=True)

classification_model.fit(x = input_train, 
                         y = y_train, 
                         epochs = 10, 
                         batch_size = 128,
                         verbose = 1,
                         validation_data=(input_test, y_test),
                         callbacks=[checkpointer])

In [ ]:
y_pred = classification_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

In [ ]:
detection_model_best = load_model('./data/weights_c.hdf5')

y_pred = detection_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

## Classification with null class
(detection and classification are performed together)

In [ ]:
[x_train, y_train, x_test, y_test, n_features, n_classes] = utils.preprocessing(subject,
                                                                                folder,
                                                                                label,
                                                                                window_size,
                                                                                stride,
                                                                                printInfo = True,
                                                                                make_binary = False,)

In [ ]:
classification_model = deeplearning.MotionClassification((window_size,n_features,1), n_classes)
classification_model.summary() # model visualization

In [ ]:
classification_model.compile(optimizer = Adam(lr=0.01), 
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

input_train = x_train.reshape(x_train.shape[0], window_size, n_features, 1)
input_test = x_test.reshape(x_test.shape[0], window_size, n_features, 1)

checkpointer = ModelCheckpoint(filepath='./data/weights_dc.hdf5', verbose=1, save_best_only=True)

classification_model.fit(x = input_train, 
                         y = y_train, 
                         epochs = 10, 
                         batch_size = 300,
                         verbose = 1,
                         validation_data=(input_test, y_test),
                         callbacks=[checkpointer])

In [ ]:
y_pred = classification_model.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))

In [ ]:
classification_model_best = load_model('./data/weights_dc.hdf5')

y_pred = classification_model_best.predict(input_test)
y_pred = np.argmax(y_pred, 1)

print(classification_report(y_test, to_categorical(y_pred)))